In [1]:
#imports
import boto3
import pyodbc
import pandas as pd
import redshift_connector
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import os
import json
from functools import reduce

In [2]:
with open('aws.json') as json_file:
    cr = json.load(json_file)

s3creds = cr['data_sources']['s3']['creds']
redcreds = cr['data_sources']['redshift']['creds']

In [3]:
#s3 connect 
s3 = boto3.resource(
    service_name='s3',
    region_name=s3creds['region_name'],
    aws_access_key_id=s3creds['aws_access_key_id'],
    aws_secret_access_key=s3creds['aws_secret_access_key']
)

tw_name = 'Tournament_Won.csv'
tl_name = 'Tournament_Lost.csv'

#s3 get data
bucket_name = s3creds['bucket']
tw_file_name = s3creds['folder_path']+tw_name
tl_file_name = s3creds['folder_path']+tl_name
bucket = s3.Bucket(bucket_name)
tw_obj = bucket.Object(key=tw_file_name)
tl_obj = bucket.Object(key=tl_file_name)
tw_response = tw_obj.get()
tl_response = tl_obj.get()
twTable = pd.read_csv(tw_response['Body'],header=3)
tlTable = pd.read_csv(tl_response['Body'],header=3)
#twTable.info()
#tlTable.info()

In [4]:
#GCP connection
KEYPATH = "gcp.json"
CREDS = service_account.Credentials.from_service_account_file(KEYPATH)
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)


q1 = """
SELECT * FROM training_dataset.tournament
"""
#q2 = """
#SELECT * FROM training_dataset.tournament_winner
#"""

view1 = client.query(q1)
#view2 = client.query(q2)
view1.result()
tournament = view1.result().to_dataframe()
#tournament_winner = view2.result().to_dataframe()
#tournament.info()
#tournament_winner.info()

In [5]:
tournament['Series'].unique()

array(['ATP250', 'Masters 1000', 'Masters', 'International', 'Grand Slam',
       'ATP500', 'International Gold', 'Masters Cup'], dtype=object)

In [6]:
intlTournamentDf = tournament.query("Series == 'International'")

In [7]:
intlTournamentDf['Series'].unique()

array(['International'], dtype=object)

In [8]:
dataframes = [intlTournamentDf,twTable]

In [10]:
merged = reduce(lambda left,right: pd.merge(left,right,on='row_order'),dataframes)

In [12]:
stdCols = ['row_order','Winner','Series']

In [22]:
winnersList = merged['Winner'].unique()
mergedStd = merged[stdCols]
uploadData = mergedStd

In [24]:
pandas_gbq.to_gbq(uploadData, destination_table = 'training_dataset.DA-13', project_id=CREDS.project_id, credentials = CREDS)